# initial

In [1]:
import numpy as np
import cv2 
import os
import mediapipe as mp
import tensorflow as tf 
from tensorflow import keras
import matplotlib.pyplot as plt
import time
import pandas as pd

2022-05-20 18:27:31.609949: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/mina/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-05-20 18:27:31.609964: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
dir_path = os.path.join("data_v2")
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
def mediapipe_detection(image,model):
    image  = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image  = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results



def draw_styled_landmarks(image,results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
    


def diff(pose,hand):
    pose_x,pose_y = pose[0].x,pose[0].y
    hand_x,hand_y = hand[0].x,hand[0].y
    
    return abs(pose_y-hand_y)
    #return abs(pose_x-hand_x) + abs(pose_y-hand_y)

    
    
def draw_rect(frame,lis):
    min_x = 1000000
    max_x=0
    min_y=1000000
    max_y=0
    
    for point in lis:
        landmark_px = mp_drawing._normalized_to_pixel_coordinates(point.x, point.y,
                                                   frame.shape[1], frame.shape[0])
        

        
        if landmark_px:

            min_x = min(landmark_px[0],min_x)
            min_y = min(landmark_px[1],min_y)
            max_x = max(landmark_px[0],max_x)
            max_y = max(landmark_px[1],max_y)
        else:
            return False
    
    
    
    Shift_X = frame.shape[1]//50
    Shift_Y = frame.shape[0]//50
    
    min_x = max(0,min_x-Shift_X)
    min_y = max(0,min_y-Shift_Y)
    max_x = min(frame.shape[0],max_x+Shift_X)
    max_y = min(frame.shape[1],max_y+Shift_Y)
    
    cv2.rectangle(frame,(min_x,min_y),(max_x,max_y),(255,0,0),10)

    return True

    

def write_text(image,text,position,size='l'):
    COLOR = (255,255,255)
    if size=='l':
        cv2.putText(image, text, position, 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, COLOR, 4, cv2.LINE_AA)
    elif size=='s':
        cv2.putText(image, text, position, 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR, 1, cv2.LINE_AA)
        
        
pose_selected_landmarks = [
    [0,2,5,11,13,15,12,14,16], # responsible for pose 
    [0,2,4,5,8,9,12,13,16,17,20], # left hand
    [0,2,4,5,8,9,12,13,16,17,20], # right hand
]



def draw_landmark_from_results(image,results):
    image_rows, image_cols, _ = image.shape
    
    original_landmarks = [
        results.pose_landmarks,
        results.left_hand_landmarks,
        results.right_hand_landmarks
    ]

    
    for shape in range(3):
        if(original_landmarks[shape]):
            lis = original_landmarks[shape].landmark
            for idx in pose_selected_landmarks[shape]:
                point = lis[idx]
                landmark_px = mp_drawing._normalized_to_pixel_coordinates(point.x, point.y,
                                                           image_cols, image_rows)

                cv2.circle(image, landmark_px, 2, (0,0,255),
                         4)



# 1 - remove first 5 frames from each video

In [6]:
input_dir_path = os.path.join("data_v2")
output_dir_path = os.path.join('data_v2_cleaned')
# signs = ["agree",'crazy','girl','look','mom','teacher','tommorow','walk','you']
signs = ["family",'friends','get_to_know','help','how_are_you','love','street','teach_me','thank_you','want']

for sign in signs:
    try:
        os.mkdir(os.path.join(output_dir_path,sign))
    except:
        pass
    videos = os.listdir(os.path.join(input_dir_path,sign))
    
    print(f'{sign} Started')

    continue_view = True
    frame_num = 0

    for video_num,video in enumerate(sorted([int(i.split('.')[0]) for i in videos])):
        video = str(video)+'.mp4'
        cap = cv2.VideoCapture(os.path.join(input_dir_path,sign,video))
        frame_num = 0
        out = cv2.VideoWriter(os.path.join(output_dir_path,sign,video),0x7634706d, 25, (640,480))
        while cap.isOpened():
            ret, frame = cap.read()
            frame_num+=1
            if not ret:
                break

            if frame_num < 6:
                continue

    #         frame,results = mediapipe_detection(frame,holistic)
    #         draw_styled_landmarks(frame,results)
    #         write_text(frame,'frame num {}'.format(frame_num),(50,40))
            # cv2.imshow('OpenCV Feed', frame)
            out.write(frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                continue_view = False
                break

        cap.release()
        out.release()

        if not continue_view:
            break
            
        if((video_num+1)%50==0):
            print("current video",video_num)
            
    print(f'{sign} finished')
    print("-"*70)
        

cv2.destroyAllWindows()

    

family Started
current video 49
current video 99
current video 149
current video 199
family finished
----------------------------------------------------------------------
friends Started
current video 49
current video 99
current video 149
current video 199
friends finished
----------------------------------------------------------------------
get_to_know Started
current video 49
current video 99
current video 149
current video 199
get_to_know finished
----------------------------------------------------------------------
help Started
current video 49
current video 99
current video 149
current video 199
help finished
----------------------------------------------------------------------
how_are_you Started
current video 49
current video 99
current video 149
current video 199
how_are_you finished
----------------------------------------------------------------------
love Started
current video 49
current video 99
current video 149
current video 199
love finished
-------------------------

In [55]:
cv2.destroyAllWindows()

# 2 - remove bad signs

In [23]:
input_dir_path = os.path.join('data_v2_cleaned')
# signs = ["agree",'crazy','girl','look','mom','teacher','tommorow','walk','you','one']
signs = ["family",'friends','get_to_know','help','how_are_you','love','street','teach_me','thank_you','want']
signs = ['love','street','teach_me','thank_you','want']

for sign in signs:

    videos = os.listdir(os.path.join(input_dir_path,sign))


    continue_view = True
    frame_num = 0

    for video_num,video in enumerate(sorted([int(i.split('.')[0]) for i in videos])):
        video = str(video)+'.mp4'
        cap = cv2.VideoCapture(os.path.join(input_dir_path,sign,video))
        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        if length < 25:
            print(sign,'/',video,sep="")
            continue
        frame_num = -1
        while cap.isOpened():
            ret, frame = cap.read()
            frame_num+=1
            if frame_num % 3 !=0:
                continue
                
            
            if not ret:
                break
                
            


            write_text(frame,'video num {}'.format(video_num),(50,40))
            write_text(frame,'frame num {}'.format(frame_num),(50,140))
            cv2.imshow('OpenCV Feed', frame)

            if cv2.waitKey(20) & 0xFF == ord('q'):
                print(sign,'/',video,sep="")
                if cv2.waitKey(100) & 0xFF == ord('t'):
                    continue_view = False
                    break

        cap.release()

        if not continue_view:
            break
    if not continue_view:
            break


    print("-"*100)


cv2.destroyAllWindows()

    

love/105.mp4
love/153.mp4
----------------------------------------------------------------------------------------------------
street/23.mp4
street/131.mp4
----------------------------------------------------------------------------------------------------
teach_me/67.mp4
teach_me/103.mp4
teach_me/176.mp4
----------------------------------------------------------------------------------------------------
thank_you/18.mp4
thank_you/154.mp4
----------------------------------------------------------------------------------------------------
want/39.mp4
----------------------------------------------------------------------------------------------------


In [52]:
cv2.destroyAllWindows()

In [15]:
cap = cv2.VideoCapture(os.path.join(input_dir_path,"thank_you","159.mp4"))

In [16]:
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print( length )

32


# 2 - get keypoints from Directory

- it does exactly as the name suggests

In [121]:
import numpy as np
import cv2 
import os
import mediapipe as mp
import tensorflow as tf 
from tensorflow import keras
import matplotlib.pyplot as plt
import time
import pandas as pd

In [122]:
def mediapipe_detection(image,model):
    image  = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image  = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results

        
    
def extract_keypoints(results,status='l'):
    if status == 'l':
        return np.array([ [res.x,res.y] for res in results.left_hand_landmarks.landmark ]).flatten()
    else:
        return np.array([ [res.x,res.y] for res in results.right_hand_landmarks.landmark ]).flatten()
    
                

def diff(pose,hand):
    pose_x,pose_y = pose[0].x,pose[0].y
    hand_x,hand_y = hand[0].x,hand[0].y
    
    return abs(pose_y-hand_y)
    #return abs(pose_x-hand_x) + abs(pose_y-hand_y)


    
def validate_hand(frame,lis):
    
    for point in lis:
        landmark_px = mp_drawing._normalized_to_pixel_coordinates(point.x, point.y,
                                                   frame.shape[1], frame.shape[0])
        if not landmark_px:
            return False


    return True

    
def add_left(data,frames,frame,results):
    condition = validate_hand(frame,results.left_hand_landmarks.landmark)
    if condition:
        data.append(extract_keypoints(results,'l'))
        frames.append(frame)
        
        

def add_right(data,frames,frame,results):
    condition = validate_hand(frame,results.right_hand_landmarks.landmark)
    if condition:
        data.append(extract_keypoints(results,'r'))
        frames.append(frame)
    

        


In [123]:
def get_key_points(images):

    data = []
    frames = []

    for i,image in enumerate(images):
        print("image num",i)

        frame = cv2.imread(os.path.join(dir_path,image))


        frame,results = mediapipe_detection(frame,holistic)


        valid_left =  results.left_hand_landmarks
        valid_right = results.right_hand_landmarks

        if valid_right and valid_left:
            left_diff = diff(results.pose_landmarks.landmark,results.left_hand_landmarks.landmark)
            right_diff = diff(results.pose_landmarks.landmark,results.right_hand_landmarks.landmark)
            if left_diff < right_diff:
                add_left(data,frames,frame,results)
            else:
                add_right(data,frames,frame,results)
        elif valid_right:
            add_right(data,frames,frame,results)
        elif valid_left:
            add_left(data,frames,frame,results)




        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame = cv2.resize(frame, (640, 480)) 
        cv2.imshow("frame",frame)

    cv2.destroyAllWindows()
    
    return data,frames

    

In [124]:
dir_path = os.path.join("..","..","data","data_letter_khaa")
images = os.listdir(dir_path)

data,frames = get_key_points(images)

image num 0
image num 1
image num 2
image num 3
image num 4
image num 5
image num 6
image num 7
image num 8
image num 9
image num 10
image num 11
image num 12
image num 13
image num 14
image num 15
image num 16
image num 17
image num 18
image num 19
image num 20
image num 21
image num 22
image num 23
image num 24
image num 25
image num 26
image num 27
image num 28
image num 29
image num 30
image num 31
image num 32
image num 33
image num 34
image num 35
image num 36
image num 37
image num 38
image num 39
image num 40
image num 41
image num 42
image num 43
image num 44
image num 45
image num 46
image num 47
image num 48
image num 49
image num 50
image num 51
image num 52
image num 53
image num 54
image num 55
image num 56
image num 57
image num 58
image num 59
image num 60
image num 61
image num 62
image num 63
image num 64
image num 65
image num 66
image num 67
image num 68
image num 69
image num 70
image num 71
image num 72
image num 73
image num 74


In [125]:
len(data),len(frames)

(72, 72)

# 3 - Test the keypoints from images

- after we got the keypoints from the frame if we are feeling that those keypoints might be wrong we can use the below code to draw the extracted keypoints with the frames

In [126]:
def draw_landmark_from_array(image,keyPoints):
    image_rows, image_cols, _ = image.shape
    
    
    for i in range(len(keyPoints)//2):
        x = keyPoints[i*2]
        y = keyPoints[i*2+1]
        if(x!=0 and y!=0): 
            landmark_px = mp_drawing._normalized_to_pixel_coordinates(x,y,
                                                       image_cols, image_rows)
            cv2.circle(image, landmark_px, 2, (0,0,255),
                     30)

                


In [127]:
for i in range(len(data)):
    
    print(f"image num {i}")
    image = frames[i]
    keypoint = data[i]

    draw_landmark_from_array(image,keypoint)

    image = cv2.resize(image, (640, 480)) 
    cv2.imshow("frame",image)
    if cv2.waitKey(500) & 0xFF == ord('q'):
        break


cv2.destroyAllWindows()

image num 0
image num 1
image num 2
image num 3
image num 4
image num 5
image num 6
image num 7
image num 8
image num 9
image num 10
image num 11
image num 12
image num 13
image num 14
image num 15
image num 16
image num 17
image num 18
image num 19
image num 20
image num 21
image num 22
image num 23
image num 24
image num 25
image num 26
image num 27
image num 28
image num 29
image num 30
image num 31
